### **Power Spectrum Computations (nbodykit):**

Compute 1D and 2D power spectra using nbodykit

In [ ]:
import sys

fastpt_path = r"/global/u2/s/samgolds/FAST-PT/"
genericio_path = (r"/global/u2/s/samgolds/DC2-analysis/contributed/"
                  "nonlinear_bias/genericio/python/")
gcrcatalogs_path = r"/global/u2/s/samgolds/gcr-catalogs/"

sys.path.append(fastpt_path)
sys.path.append(genericio_path)
sys.path.append(gcrcatalogs_path)

import GCRCatalogs
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import pyccl
import sys

# Import nbodykit associated libraries
from nbodykit.lab import *
from nbodykit import style
from nbodykit.source.catalog import CSVCatalog

from scipy.constants import speed_of_light
from matplotlib.ticker import NullFormatter

plt.style.use(style.notebook)
sns.set(style='ticks')

%config IPCompleter.greedy = True
%config InlineBackend.figure_format = 'retina'


# Load catalog and galaxy positions
cat_str = "baseDC2_snapshot_z1.01_v0.1"
cat_str = "baseDC2_snapshot_z0.15_v0.1_small"
cat = GCRCatalogs.load_catalog(cat_str)

def get_catalog_cosmology(cat_str):
    """
    Helper function to return catalog cosmology object and redshift
    """
    
    cat = GCRCatalogs.load_catalog(cat_str)
    
    return cat.cosmology, cat.redshift

COSMO, Z_RED_SHIFT = get_catalog_cosmology(cat_str)
COSMO_CCL = pyccl.Cosmology(h=COSMO.h, sigma8=COSMO.sigma8,
                            Omega_g=COSMO.Ogamma0, Omega_k=COSMO.Ok0,
                            Omega_c=COSMO.Odm0, Omega_b=COSMO.Ob0,
                            n_s=COSMO.n_s, Neff=COSMO.Neff)

# Compute sigma chi
H0 = 71.0
SIGMA_Z = 0.05*(1+Z_RED_SHIFT)
SCALE_FACTOR = 1/(1+Z_RED_SHIFT)
H_Z = pyccl.background.h_over_h0(COSMO_CCL, SCALE_FACTOR)*H0

SIGMA_CHI = (speed_of_light/10**3)/H_Z*SIGMA_Z

cat_vals = cat.get_quantities(["position_x", "position_y", "position_z",
                                   "Mag_true_r_lsst_z0"])

# Galaxy positions in h^{-1}Mpc
x_data = cat_vals["position_x"]*H0/100
y_data = cat_vals["position_y"]*H0/100
z_data = cat_vals["position_z"]*H0/100


def mask_positions(q1_data, q2_data, qr_data):
    """
    A helper function which applies a gaussian mask to select galaxies based on 
    their radial position data. q1 and q2 refer to perpendicular positions (e.g. if 
    Z is radial then q1 = X and q2 = Y)

    Parameters:
    q1_data (float array): np array containing first perpendicular positions of galaxies in Mpc
    q2_data (float array): np array containing second perpendicular positions of galaxies in Mpc
    radial_data (float array): np array containing radial positions of galaxies in Mpc

    Returns:
    q1_masked (float array): np array containing x positions of masked galaxies
    in Mpc
    q2_masked (float array): np array containing y positions of masked galaxies 
    in Mpc
    """
    
    # Apply Gaussian kernel with mean z_bar and std_dev = sigma_chi
    qr_bar = np.mean(qr_data)
    
    cutoffs = np.random.uniform(0, 1, len(qr_data))
    
    mask_ind = np.where(cutoffs < 
                        np.exp(-(qr_data-qr_bar)**2/(2*SIGMA_CHI**2)))[0]

    q1_masked = q1_data[mask_ind]
    q2_masked = q2_data[mask_ind]
    qr_masked = qr_data[mask_ind]

    return q1_masked, q2_masked, qr_masked


x_data, y_data, z_data = mask_positions(x_data, y_data, z_data)

#### **Numerically Compute 2D Power Spectra:**

Define functions to numerically compute 2D power spectra using FFT.

In [ ]:
def calculate_auto_pow_spec_2D(q1_data, q2_data,  N, bad_ind):
    """ 
    Calculates the 2D auto power spectrum from data with N*N grid. Removes zero
    order Fourier mode.

    Parameters:
    q1_data (np float array): Array containing q1 positions of galaxies in Mpc
    q2_data (np float array): Array containing q2 positions of galaxies in Mpc
    N (int): Integer representing the grid size for the power spec calculation
    bad_ind (np int array): Array containing the indicies of any erroneous power
    spectra data points

    Returns:
    k_vals (np float array): array containing the wave numbers in Mpc^-1
    p_k (np float array): array containing the returned power spectrum values in
    Mpc^2
    """

    q1_min = np.floor(np.min(q1_data))
    q1_max = np.ceil(np.max(q1_data))
    q1_width = q1_max-q1_min
    
    q2_min = np.floor(np.min(q2_data))
    q2_max = np.ceil(np.max(q2_data))
    q2_width = q2_max-q2_min

    # Initialize Grid
    grid_matrix = np.histogram2d(q1_data, q2_data, N)[0]

    # Get the number of galaxies and set amount of data points per bin
    n_g = len(q1_data)

    delta_q1 = q1_width/N
    delta_q2 = q2_width/N

    # Convert grid to represent over_density
    p_bar = n_g/N**2  # No. galaxies expected per bin
    p_bar_mat = p_bar*np.ones_like(grid_matrix)

    delta = 1/p_bar*(grid_matrix-p_bar_mat)

    # Perform fourier transform
    delta_k = np.fft.fft2(delta)

    # Calculate power spectrum
    pow_spec = np.real(delta_k*np.conj(delta_k))/(N**2*N**2/(q1_width*q2_width))
    
    # Set the flawed wavenumbers to specified key for removal at end of computation
    bad_ind_key = 888888
    
    for ind in bad_ind:
        pow_spec[ind][0] = bad_ind_key
        
    p_k = pow_spec.flatten()
    
    # Get frequency values
    f_value = np.fft.fftfreq(N)
    kq1_mat = np.outer(np.ones(N), 2.0*np.pi*f_value/delta_q1)
    kq2_mat = np.outer(2.0*np.pi*f_value/delta_q2, np.ones(N))

    # Construct matrix of wave numbers
    k_mat = np.sqrt(kq1_mat**2+kq2_mat**2)
    
    k_vals = k_mat.flatten()
    assert(len(k_vals) == len(p_k))
    
    # Remove flawed wavenumbers from final values    
    pow_ind = np.where(p_k != bad_ind_key)[0]
    k_vals = k_vals[pow_ind]
    p_k = p_k[pow_ind]
    
    assert(len(np.where(p_k==bad_ind_key)[0]) == 0)
    
    # Remove zero order mode on return
    return k_vals[1:], p_k[1:]


def average_pow_spec(k_vals, p_vals, n_bins):
    """
    Averages power spectrum into n_bins based on k
    
    Parameters:
    k_vals (np float array): array containing the unaverged wave numbers in 
    Mpc^-1
    p_k (np float array): array containing the unaveraged power spectrum values in
    Mpc^2
    n_bins (int): integer representing the number of bins to average values into

    Returns:
    averaged_k (np float array): array containing the averaged wave numbers in
    Mpc^-1
    averaged_p (np float array): array containing the averged power spectrum values in
    Mpc^2
    n_modes (np int array): array containing the number of Fourier modes in each bin of 
    averaged_k/averaged_p
    """

    averaged_p, averaged_k, binnumbers = scipy.stats.binned_statistic(k_vals, p_vals, 'mean', bins=n_bins)
    
    # Compute average wave number of each bin
    averaged_k = averaged_k+(averaged_k[1]-averaged_k[0])/2
    
    # Calculate n_modes from bin number
    n_modes = np.bincount(binnumbers)[1:]
    
    return averaged_k[:-1], averaged_p, n_modes

#### **Compare Numerical 2D Power Spectra with nbodykit:**

Compare results of computing galaxy auto-power spectrum on x and y data.

In [ ]:
# Compute 2D Power spectrum using numerical algorithm
k_num_total, p_k_num_total = calculate_auto_pow_spec_2D(x_data, y_data,  N=10000, bad_ind=[])
k_num, p_k_num, n_modes_num = average_pow_spec(k_num_total, p_k_num_total, 512) # Average

# Compute 2D Power spectrum using nbodykit

# Compute using CSV data
positional_data = np.vstack((x_data, y_data, z_data)).T
names =['x', 'y', 'z']
# numpy.savetxt('csv-positions.txt', positional_data, fmt='%.7e')

# read the data
f = CSVCatalog('csv-positions.txt', names)

# combine x, y, z to Position, and add boxsize
f['Position'] = f['x'][:, None] * [1, 0, 0] + f['y'][:, None] * [0, 1, 0] + f['z'][:, None] * [0, 0, 1]
f.attrs['BoxSize'] = cat.box_size

mesh_f = f.to_mesh(Nmesh=256)
r_3d_f = FFTPower(mesh_f, mode='1d', dk=0.005, kmin=0.01)
r_2d_f = ProjectedFFTPower(mesh_f, dk=0.0005, kmin=0.001, axes=[0,1])

P2D_f = r_2d_f.power
P3D_f = r_3d_f.power

# Compute using ArrayMesh
xyz_array = np.histogramdd((x_data,y_data,z_data), bins=256)[0]
mesh = ArrayMesh(xyz_array, BoxSize=cat.box_size)

r_3d = FFTPower(mesh, mode='1d', dk=0.0005, kmin=0.001)
r_2d = ProjectedFFTPower(mesh, dk=0.0005, kmin=0.001, axes=[0,1])

P2D = r_2d.power
P3D = r_3d.power

# Plot results
x_width = np.ptp(x_data)
y_width = np.ptp(y_data)

N_BAR_G = len(x_data)/(x_width*y_width)

plt.figure(figsize=(8,8))

plt.loglog(P2D['k'], P2D['power'].real, label="ArrayMesh")
plt.loglog(P2D_f['k'], P2D_f['power'].real, label="CSV")
plt.loglog(k_num, p_k_num, '.', label=r"$P_\mathrm{2D}$ (direct)")
plt.axhline(1/N_BAR_G, linestyle=":", color="k", linewidth =0.8)
plt.legend(frameon=False)
plt.title("Galaxy 2D Auto-Power Spectrum Comparison", fontweight="bold")
plt.xlabel(r"$k$ [$h \ \mathrm{Mpc}^{-1}$]")
plt.ylabel(r"$P(k)$ [$h^{-3}\mathrm{Mpc}^3$]")

plt.savefig("nbodykit_pow_spec.png", dpi=250)